In [8]:
from pathlib import Path

import pandas as pd

DATA_DIR = Path(r'C:\Python\projects\CGPAPrediction\data\final_grade_prediction_dataset')

all_data = pd.concat([pd.read_csv(DATA_DIR / 'train.csv'), pd.read_csv(DATA_DIR / 'val.csv'), pd.read_csv(
    DATA_DIR / 'test.csv')], axis=0)

In [9]:
len(all_data['Student ID'].unique())

2423

In [10]:
(all_data['Semester'] == 4).sum()

np.int64(14817)

In [11]:
df = all_data

In [13]:
print("=== Basic Info ===")
print(df.info())

=== Basic Info ===
<class 'pandas.core.frame.DataFrame'>
Index: 77482 entries, 0 to 11591
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Student ID                   77482 non-null  int64  
 1   Student Full Name            77482 non-null  int64  
 2   Educational Program / Major  77482 non-null  int64  
 3   Payment Type                 77482 non-null  int64  
 4   Funding Type                 77482 non-null  int64  
 5   Language of Instruction      77482 non-null  int64  
 6   Institute                    77482 non-null  int64  
 7   Study Mode                   77482 non-null  int64  
 8   Group                        77482 non-null  int64  
 9   Course                       77482 non-null  int64  
 10  Semester                     77482 non-null  int64  
 11  Instructor Full Name         77482 non-null  int64  
 12  Instructor Department        77482 non-null  int64  
 13  St

In [14]:
print("\n=== General Statistics (numerical) ===")
print(df.describe().T)


=== General Statistics (numerical) ===
                               count           mean          std      min  \
Student ID                   77482.0  143458.364330  3192.830929  16216.0   
Student Full Name            77482.0    1191.565086   706.984799      0.0   
Educational Program / Major  77482.0      15.146821    13.804775      0.0   
Payment Type                 77482.0       0.498632     0.500001      0.0   
Funding Type                 77482.0       1.401332     1.298606      0.0   
Language of Instruction      77482.0       0.387664     0.487220      0.0   
Institute                    77482.0       3.906998     3.538514      0.0   
Study Mode                   77482.0       0.285744     0.865421      0.0   
Group                        77482.0      96.736339    75.373735      0.0   
Course                       77482.0     264.837898   345.177838      0.0   
Semester                     77482.0       1.912599     1.427332      0.0   
Instructor Full Name         77482.0

In [12]:


print("\n=== Categorical Features (counts) ===")
for col in ['Educational Program / Major', 'Payment Type', 'Funding Type',
            'Language of Instruction', 'Institute', 'Study Mode', 'Group',
            'Course', 'Semester', 'Instructor Department',
            'Student Gender', 'Instructor Gender']:
    print(f"\n--- {col} ---")
    print(df[col].value_counts(normalize=True).head(10))

=== Basic Info ===
<class 'pandas.core.frame.DataFrame'>
Index: 77482 entries, 0 to 11591
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Student ID                   77482 non-null  int64  
 1   Student Full Name            77482 non-null  int64  
 2   Educational Program / Major  77482 non-null  int64  
 3   Payment Type                 77482 non-null  int64  
 4   Funding Type                 77482 non-null  int64  
 5   Language of Instruction      77482 non-null  int64  
 6   Institute                    77482 non-null  int64  
 7   Study Mode                   77482 non-null  int64  
 8   Group                        77482 non-null  int64  
 9   Course                       77482 non-null  int64  
 10  Semester                     77482 non-null  int64  
 11  Instructor Full Name         77482 non-null  int64  
 12  Instructor Department        77482 non-null  int64  
 13  St

In [17]:
len(all_data['Instructor Full Name'].unique())

562

In [18]:
all_data['Final Grade'].map(lambda x: 1 if x < 50 else 0).sum()

np.int64(236)

In [19]:
all_data['Final Grade'].map(lambda x: 1 if x < 50 else 0).sum()/len(all_data)

np.float64(0.0030458687178957695)

In [23]:
vc = all_data.groupby(['Semester', 'Student ID']).size().value_counts()

In [26]:
mean_subjects = (vc.index.to_numpy() * vc.values).sum() / vc.values.sum()
print(mean_subjects)

6.395542715641766


In [27]:
inputs_description = {'Educational Program / Major': 'embedding', 'Payment Type': 'one-hot',
                      'Funding Type': 'one-hot', 'Language of Instruction': 'one-hot',
                      'Institute': 'one-hot', 'Study Mode': 'one-hot', 'Course': 'embedding',
                      'Instructor Full Name': 'embedding', 'Instructor Department': 'embedding',
                      'Student Gender': 'one-hot', 'Instructor Gender': 'one-hot', 'Rating': 'number',
                      'Midterm 1': 'number', 'Midterm 2': 'number'}

In [29]:
cat_f = [k for k, v in inputs_description.items() if v != 'number']

In [31]:
res = {k: len(all_data[k].unique()) for k in cat_f}

In [32]:
res

{'Educational Program / Major': 58,
 'Payment Type': 2,
 'Funding Type': 9,
 'Language of Instruction': 2,
 'Institute': 13,
 'Study Mode': 7,
 'Course': 1533,
 'Instructor Full Name': 562,
 'Instructor Department': 74,
 'Student Gender': 2,
 'Instructor Gender': 2}

In [34]:
def grade_to_cat(x):
    if x < 70:
        return 0
    elif x < 90:
        return 1
    return 2

In [43]:
vc = all_data.loc[all_data['Semester'] == 4, 'Final Grade'].apply(grade_to_cat).value_counts()

In [44]:
vsp = {k: v/vc.sum()*100 for k, v in vc.items()}

In [45]:
vsp

{1: np.float64(49.08550988729162),
 2: np.float64(36.66734156711885),
 0: np.float64(14.247148545589525)}

In [47]:
len(all_data.loc[all_data['Semester'] == 4])

14817

In [48]:
train_data = pd.read_csv(DATA_DIR / 'train.csv')

In [49]:
len(train_data.loc[train_data['Semester'] == 4])

10373